<a href="https://colab.research.google.com/github/puraminy/mini_proj2/blob/master/BAM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class BAM(object):
    def __init__(self, data):
        self.pairs = data
        m1 = len(self.pairs[0][1])
        m2 = len(self.pairs[0][0])
        # create weight matrix
        self.W = np.zeros((m2,m1))
        for p in self.pairs:
          X = p[0]
          Y = p[1]
          self.W += np.outer(X, Y)

    def get_assoc(self, A):
        A = np.asarray(A)
        y=np.dot(A.T,self.W)
        y[y<0]=-1
        y[y>=0]=1
        return y

  

import matplotlib.pyplot as plt

def print_formatted(row_list,col_list , mat):
  row_format ="{:>10}" * (len(col_list) + 1)
  print(row_format.format("", *col_list))
  for item, row in zip(row_list, mat):
      print(row_format.format(item, *row))

def draw_img(ax, X):
  ax.imshow(np.reshape(X,[5,3]))
  
def test_noise (inp, te_n, n_n): #inp:input vector te_n: number of noisy samples for test. n_n: number of noisy elements
  test = []
  for i in range(te_n):
    rnd = random.sample(range(len(inp)), n_n)
    noise = np.asarray(inp)
    for i in rnd:
      noise[i] = -1*noise[i]
    test.append(noise)

  test=np.asarray(test)
  test = np.unique(test,axis=0)

  return test


In [71]:
import numpy as np
import random

A = [-1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1]
tA = [-1, -1,-1] 
B = [1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1]
tB = [-1, -1, 1]
C = [-1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1]
tC = [-1,1,-1] 
D = [1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1]
tD = [-1,1,1]
E = [1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1]
tE = [1,-1,-1]
F =[1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, -1]
tF = [1, -1, 1]
G = [-1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1]
tG = [1, 1, -1]
H = [1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1]
tH = [1, 1, 1]

# tA = [-1, 1] 
# tC = [1, 1] 
letters ={"A" : A,"B" : B, "C" : C,"D":D, "E":E,"F":F,"G":G,"H":H}
targets ={"A" : tA,"B": tB, "C" : tC, "D" : tD, 
           "E" : tE,"F": tF, "G" : tG, "H" : tH}

# fig, axes = plt.subplots(2, 4)
# plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
# for i in range(4):
#   draw_img(axes[0,i], X[i])
#   xlabel = Y[i]; axes[0,i].set_xlabel(xlabel)
#   draw_img(axes[1,i], X[i+4]) 
#   xlabel = Y[i+4]; axes[1,i].set_xlabel(xlabel)

ds=[]

ds.append(dict((k, letters[k]) for k in ('A','C'))) #0
ds.append(dict((k, letters[k]) for k in ('A','B'))) 
ds.append(dict((k, letters[k]) for k in ('E','D'))) #2
ds.append(dict((k, letters[k]) for k in ('D','C')))
ds.append(dict((k, letters[k]) for k in ('A','B','C'))) #4
ds.append(dict((k, letters[k]) for k in ('F','C','H')))
ds.append(dict((k, letters[k]) for k in ('A','B','C','D','E')))
ds.append(dict((k, letters[k]) for k in ('A','B','C','D','E','F')))
ds.append(dict((k, letters[k]) for k in ('A','B','C','D','E','F','G','H'))) #8


let_list = [v for v in letters.keys()]


acc_mat = np.full((len(ds), 8),-1)
ds_index = 0

sel_ds = [ds[8]]

for data in sel_ds:
  print("==========")
  print(data.keys())
  pairs = []

  for key,value in data.items():
    y = targets[key]
    pairs.append([value, y])

  bam = BAM(pairs)
  
  for b in range(9):
    for key,letter in data.items():
      target = targets[key]
      test_data=test_noise(letter, 100, b)
      success = 0
      for i,noisy_data in enumerate(test_data):
        predicted = bam.get_assoc(noisy_data)
        if np.array_equal(target, predicted):
          success +=1

#       print(f"Accuracy of {key}= {success}")
      let_index = let_list.index(key)
      acc_mat[b, let_index] = success

    ds_index+=1
  
print_formatted(range(len(ds)), let_list, acc_mat)
    


dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
                   A         B         C         D         E         F         G         H
         0         0         1         0         1         1         1         1         0
         1         3        15         2        11        13        13        12         4
         2        13        60        15        35        37        49        42        22
         3         8        68        21        28        46        64        48        22
         4         8        54        31        41        37        51        49        22
         5         6        42        19        25        21        34        36        12
         6         7        27        25        17        16        23        21         4
         7         7        14        11         6         9        17        18         9
         8         5        11        18         6         9         8        22         4
